In [3]:
import warnings
warnings.filterwarnings("ignore")

import DataUtils.MATH as MATH
from hfUtils import *
from ModelUtils import *

import numpy as np

import torch

In [4]:
tokenizer, model = load_model_and_tokenizer('meta-llama/Llama-3.2-1B-Instruct')

In [5]:
MATH_inputs_chat, MATH_Answers_chat = MATH.get_test_data_chat_template()

MATH_inputs_plain, MATH_Answers_plain = MATH.get_test_data_plain()

tokenized_MATH_inputs_chat = tokenize_chat(MATH_inputs_chat, tokenizer)
tokenized_MATH_inputs_plain = tokenize_plain(MATH_inputs_plain, tokenizer)

100%|██████████| 5000/5000 [00:01<00:00, 4747.15it/s]


In [9]:
print(MATH_inputs_chat[0])
print("AAAAAAA \n\n")
print(MATH_inputs_plain[0])

[{'role': 'system', 'content': 'You will be given instructions that describes a task. Write a response that appropriately completes the request. Think step by step. Decorate the final answer with \\boxed{}'}, {'role': 'user', 'content': 'Find the matrix $\\mathbf{M}$ such that\n\\[\\mathbf{M} \\begin{pmatrix} 1 & -2 \\\\ 1 & 4 \\end{pmatrix} = \\begin{pmatrix} 6 & 0 \\\\ 0 & 6 \\end{pmatrix}.\\]'}]
AAAAAAA 


Below is an instruction that describes a task. Write a response that appropriately completes the request. Think step by step. Decorate the final answer with \boxed{}

### Instruction:
Find the matrix $\mathbf{M}$ such that
\[\mathbf{M} \begin{pmatrix} 1 & -2 \\ 1 & 4 \end{pmatrix} = \begin{pmatrix} 6 & 0 \\ 0 & 6 \end{pmatrix}.\]

### Response: Let's think step by step.


In [42]:
problem_prompt = (
    "You will be given instructions that describes a task. Write a response that appropriately completes the request. Think step by step. Decorate the final answer with \\boxed{}",
    #"Below is an instruction that describes a task. Write a response that appropriately completes the request. Think step by step. Decorate the final answer with \\boxed{}",
    "Write a consice response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Response: Let's think step by step."
    #"Below is an instruction that describes a task. "
    #"Write a response that appropriately completes the request.\n\n"
    #"### Instruction:\n{instruction}\n\n### Response: Let's think step by step."
)

def get_input_kwargs(tokenizer):

    max_new_tokens = 2048
    num_beams = 1
    do_sample = False
    top_p = 1.0
    
    return {"max_new_tokens": max_new_tokens, "do_sample": do_sample, "top_p": top_p}

#    Widen:
#    stop_tokens = ["Instruction:", "Instruction", "Response:", "Response"]
#    sampling_params = SamplingParams(temperature=0.0, top_p=1, max_tokens=2048, stop=stop_tokens)

In [ ]:
N = 10
results = run_model(tokenized_MATH_inputs_chat[:N], model, tokenizer, **get_input_kwargs(tokenizer))

answers = MATH.extract_answers(results)

c = 0
for i, answer in enumerate(answers):
    if answer == MATH_Answers_chat[i]:
        c += 1
    print("answer: ", MATH_Answers_chat[i])
    print("model: ", answer)
    print("true answers: ", c,"total answers: ", i)

print("accuracy: ", c/N)

  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 10%|█         | 1/10 [00:50<07:34, 50.53s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 20%|██        | 2/10 [01:00<03:34, 26.85s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
 30%|███       | 3/10 [02:08<05:18, 45.53s/it]The attention mask and the pad token id were not set. As a consequence, yo

answer:  \begin{pmatrix} 4 & 2 \\ -1 & 1 \end{pmatrix}
model:  \begin{pmatrix} 3 & 0 \\ 0 & 3 \end{pmatrix}
true answers:  0 total answers:  0
answer:  \pi
model:  To compute $\arccos (-1)$, we need to find the angle whose cosine is $-1$. 

We know that $\cos(\frac{\pi}{2}) = 0$, $\cos(\frac{3\pi}{2}) = 0$, and $\cos(\frac{4\pi}{2}) = \cos(2\pi) = 1$. 

Since $\cos(\frac{\pi}{2}) = 0$ and $\cos(\frac{3\pi}{2}) = 0$, we can conclude that $\arccos(-1) = \frac{\pi}{2}$ or $\arccos(-1) = \frac{3\pi}{2}$.
true answers:  0 total answers:  1
answer:  0
model:  To find $\sin x + \sin 2x + \sin 3x$, we can use the given equation $\cos x + \cos 2x + \cos 3x = 3$ and the trigonometric identity $\cos A + \cos B = 2\cos\left(\frac{A+B}{2}\right)\cos\left(\frac{A-B}{2}\right)$.

Applying the identity to the given equation, we get:

$$\cos x + \cos 2x + \cos 3x = 3 \Rightarrow 2\cos\left(\frac{x+2x}{2}\right)\cos\left(\frac{x-2x}{2}\right) + 2\cos\left(\frac{3x+3x}{2}\right)\cos\left(\frac{3x-3x}{2}\

In [45]:
N = 100
results = run_model(tokenized_MATH_inputs_plain[:N], model, tokenizer, **get_input_kwargs(tokenizer))

answers = MATH.extract_answers(results)

  0%|          | 0/100 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  1%|          | 1/100 [00:10<16:37, 10.07s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  2%|▏         | 2/100 [00:16<12:32,  7.68s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
  3%|▎         | 3/100 [00:37<22:26, 13.88s/it]The attention mask and the pad token id were not set. As a consequence

In [47]:
import re

def normalize_latex(latex_str):
    """
    Normalize LaTeX string by:
    1. Removing extra whitespace
    2. Standardizing common variations
    3. Removing comments
    """
    # Remove LaTeX comments
    latex_str = re.sub(r'%.*$', '', latex_str, flags=re.MULTILINE)
    
    # Remove extra whitespace
    latex_str = re.sub(r'\s+', ' ', latex_str.strip())
    
    # Standardize spacing around operators
    latex_str = re.sub(r'\s*([=+\-*/])\s*', r'\1', latex_str)
    
    # Standardize curly braces
    latex_str = re.sub(r'{\s+', '{', latex_str)
    latex_str = re.sub(r'\s+}', '}', latex_str)
    
    # Standardize square brackets
    latex_str = re.sub(r'\[\s+', '[', latex_str)
    latex_str = re.sub(r'\s+\]', ']', latex_str)
    
    # Standardize parentheses
    latex_str = re.sub(r'\(\s+', '(', latex_str)
    latex_str = re.sub(r'\s+\)', ')', latex_str)
    
    return latex_str

def compare_latex(latex1, latex2, normalize=True):
    """
    Compare two LaTeX strings.
    
    Args:
        latex1 (str): First LaTeX string
        latex2 (str): Second LaTeX string
        normalize (bool): Whether to normalize the strings before comparison
        
    Returns:
        bool: True if the strings are equivalent, False otherwise
    """
    if normalize:
        latex1 = normalize_latex(latex1)
        latex2 = normalize_latex(latex2)
    
    return latex1 == latex2

In [48]:
c = 0
for i, answer in enumerate(answers):
    c += compare_latex(answer, MATH_Answers_plain[i])
    #print("answer: ", MATH_Answers_plain[i])
    #print("model: ", answer)
    #print("true answers: ", c,"total answers: ", i+1)

print("accuracy: ", c/N)

accuracy:  0.08


In [49]:
c = 0
for i, answer in enumerate(answers):
    if answer == MATH_Answers_plain[i]:
        c += 1

print("accuracy plain: ", c/N)

accuracy plain:  0.08


In [31]:
import random

answer_ = answer

for i in range(3):
    j = random.randint(0, 100)
    print(j)

    print("answer: ", MATH_Answers_chat[j])
    print("model: ", answer[j])
    print("results: ", results[j])
    print()

84
answer:  \sec x
model:  i
results:   

To simplify the given expression, we can start by finding a common denominator for the two fractions. The common denominator will be $(\cos x)(1 + \sin x)$.

\[\frac{\sin x}{\cos x} + \frac{\cos x}{1 + \sin x} = \frac{\sin x(1 + \sin x)}{\cos x(1 + \sin x)} + \frac{\cos x \cos x}{(1 + \sin x)\cos x}.\]

Now, we can combine the fractions by adding the numerators.

\[\frac{\sin x(1 + \sin x) + \cos^2 x}{\cos x(1 + \sin x)}.\]

We can simplify the numerator by using the identity $\sin^2 x + \cos^2 x = 1$. 

\[\frac{\sin x(1 + \sin x) + 1 - \sin^2 x}{\cos x(1 + \sin x)}.\]

Now, we can factor the numerator.

\[\frac{(\sin x + 1)(1 + \sin x) - \sin^2 x}{\cos x(1 + \sin x)}.\]

We can simplify the numerator further by expanding the terms.

\[\frac{\sin x + 1 + \sin^2 x - \sin^2 x}{\cos x(1 + \sin x)}.\]

Now, we can simplify the numerator by canceling out the $\sin^2 x$ terms.

\[\frac{\sin x + 1}{\cos x(1 + \sin x)}.\]

### Final Answer:
\boxed{\fra

In [55]:
j = 84

def extract_answer(response):
    a = response.split("boxed{")[-1]
    a = find_first_unopened_closing_brace(a)
    return a

def find_first_unopened_closing_brace(a):
    x = 0
    for i, char in enumerate(a):
        if char == '{':
            x += 1 
        elif char == '}':
            x -= 1
        if x == -1:
            return a[:i]
        
    return a
for i in range(10,20):
    print(i)
    print(answer[i])
    print(extract_answer(results[i]))
    print(MATH_Answers_plain[i])
    print(compare_latex(extract_answer(results[i]), MATH_Answers_plain[i]))

10
e
(5.5,-1,1)
\left( \frac{11}{2}, -1, 1 \right)
False
11
 
 

To compute $\operatorname{arccot} (-1)$, we need to find the angle whose cotangent is $-1$. Recall that $\cot \theta = \frac{\cos \theta}{\sin \theta}$. Therefore, we need to find the angle whose cosine is $-1$ and whose sine is $0$.

### Step 1: Recall the unit circle and the definition of cotangent.
The unit circle is a circle with a radius of $1$ centered at the origin of the coordinate plane. The cotangent of an angle $\theta$ is defined as the ratio of the cosine of $\theta$ to the sine of $\theta$. Therefore, we are looking for an angle whose cosine is $-1$ and whose sine is $0$.

### Step 2: Find the angle whose cosine is $-1$ and whose sine is $0$.
The angle whose cosine is $-1$ is $\frac{3\pi}{2}$, and the angle whose sine is $0$ is $0$.

### Step 3: Find the angle whose cotangent is $-1$.
Since $\cot \theta = \frac{\cos \theta}{\sin \theta}$, we have $\cot \frac{3\pi}{2} = \frac{\cos \frac{3\pi}{2}}{\sin \frac{3

In [57]:
c = 0
for i, result in enumerate(results):
    c += compare_latex(extract_answer(result), MATH_Answers_plain[i])
    #print("answer: ", MATH_Answers_plain[i])
    #print("model: ", answer)
    #print("true answers: ", c,"total answers: ", i+1)
    if compare_latex(extract_answer(result), MATH_Answers_plain[i]):
        print(i)
        print(answers[i])
        print(MATH_Answers_plain[i])
        print(extract_answer(result))


print("accuracy: ", c/N)

7
\frac{1}{2}
\frac{1}{2}
\frac{1}{2}
8
16
16
16
15
5
5
5
19
-3
-3
-3
57
-\frac{33}{65}
-\frac{33}{65}
-\frac{33}{65}
89
30
30
30
90
\frac{\sqrt{3}}{3}
\frac{\sqrt{3}}{3}
\frac{\sqrt{3}}{3}
97
\frac{21}{20}
\frac{21}{20}
\frac{21}{20}
accuracy:  0.08


In [75]:
def fix_fracs(string):
    substrs = string.split("\\frac")
    new_str = substrs[0]
    if len(substrs) > 1:
        substrs = substrs[1:]
        for substr in substrs:
            new_str += "\\frac"
            if substr[0] == "{":
                new_str += substr
            else:
                try:
                    assert len(substr) >= 2
                except AssertionError:
                    return string
                a = substr[0]
                b = substr[1]
                if b != "{":
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}{" + b + "}" + post_substr
                    else:
                        new_str += "{" + a + "}{" + b + "}"
                else:
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}" + b + post_substr
                    else:
                        new_str += "{" + a + "}" + b
    string = new_str
    return string


def fix_a_slash_b(string):
    if len(string.split("/")) != 2:
        return string
    a = string.split("/")[0]
    b = string.split("/")[1]
    try:
        a = int(a)
        b = int(b)
        assert string == "{}/{}".format(a, b)
        new_string = "\\frac{" + str(a) + "}{" + str(b) + "}"
        return new_string
    except AssertionError:
        return string


def remove_right_units(string):
    # "\\text{ " only ever occurs (at least in the val set) when describing units
    if "\\text{ " in string:
        splits = string.split("\\text{ ")
        assert len(splits) == 2
        return splits[0]
    else:
        return string


def fix_sqrt(string):
    if "\\sqrt" not in string:
        return string
    splits = string.split("\\sqrt")
    new_string = splits[0]
    for split in splits[1:]:
        if split[0] != "{":
            a = split[0]
            new_substr = "\\sqrt{" + a + "}" + split[1:]
        else:
            new_substr = "\\sqrt" + split
        new_string += new_substr
    return new_string


def strip_string(string):
    # linebreaks
    string = string.replace("\n", "")

    # remove inverse spaces
    string = string.replace("\\!", "")

    # replace \\ with \
    string = string.replace("\\\\", "\\")

    # replace tfrac and dfrac with frac
    string = string.replace("tfrac", "frac")
    string = string.replace("dfrac", "frac")

    # remove \left and \right
    string = string.replace("\\left", "")
    string = string.replace("\\right", "")

    # Remove circ (degrees)
    string = string.replace("^{\\circ}", "")
    string = string.replace("^\\circ", "")

    # remove dollar signs
    string = string.replace("\\$", "")

    # remove units (on the right)
    string = remove_right_units(string)

    # remove percentage
    string = string.replace("\\%", "")
    string = string.replace("\%", "")  # noqa: W605

    # " 0." equivalent to " ." and "{0." equivalent to "{." Alternatively, add "0" if "." is the start of the string
    string = string.replace(" .", " 0.")
    string = string.replace("{.", "{0.")
    # if empty, return empty string
    if len(string) == 0:
        return string
    if string[0] == ".":
        string = "0" + string

    # to consider: get rid of e.g. "k = " or "q = " at beginning
    if len(string.split("=")) == 2:
        if len(string.split("=")[0]) <= 2:
            string = string.split("=")[1]

    # fix sqrt3 --> sqrt{3}
    string = fix_sqrt(string)

    # remove spaces
    string = string.replace(" ", "")

    # \frac1b or \frac12 --> \frac{1}{b} and \frac{1}{2}, etc. Even works with \frac1{72} (but not \frac{72}1). Also does a/b --> \\frac{a}{b}
    string = fix_fracs(string)

    # manually change 0.5 --> \frac{1}{2}
    if string == "0.5":
        string = "\\frac{1}{2}"

    # NOTE: X/Y changed to \frac{X}{Y} in dataset, but in simple cases fix in case the model output is X/Y
    string = fix_a_slash_b(string)

    return string


def is_equiv(str1, str2, verbose=False):
    if str1 is None and str2 is None:
        print("WARNING: Both None")
        return True
    if str1 is None or str2 is None:
        return False

    try:
        ss1 = strip_string(str1)
        ss2 = strip_string(str2)
        # pdb.set_trace()
        if verbose:
            print(ss1, ss2)
        return ss1 == ss2
    except Exception:
        return str1 == str2

In [76]:
is_equiv(extract_answer(results[10]), MATH_Answers_plain[10])

False

In [77]:
extract_answer(results[10])

'(5.5,-1,1)'

In [78]:
MATH_Answers_plain[10]

'\\left( \\frac{11}{2}, -1, 1 \\right)'

In [84]:
c = 0
for i, answer in enumerate(answers):
        
    extract_ans_temp = result[i]
    extract_ans_temp = extract_ans_temp.strip()
    if len(extract_ans_temp) > 0 and extract_ans_temp[-1] == '.':
        extract_ans = extract_ans_temp[0:-1]
    else:
        extract_ans = extract_ans_temp
    print(i, extract_ans)
    c += is_equiv(extract_ans, MATH_Answers_plain[i])

print("accuracy plain: ", c/N)

0 
1 W
2 e
3 
4 c
5 a
6 n
7 
8 u
9 s
10 e
11 
12 t
13 h
14 e
15 
16 g
17 i
18 v
19 e
20 n
21 
22 e
23 x
24 p
25 r
26 e
27 s
28 s
29 i
30 o
31 n
32 s
33 
34 f
35 o
36 r
37 
38 $
39 P
40 $
41 
42 a
43 n
44 d
45 
46 $
47 Q
48 $
49 
50 t
51 o
52 
53 f
54 i
55 n
56 d
57 
58 a
59 
60 r
61 e
62 l
63 a
64 t
65 i
66 o
67 n
68 s
69 h
70 i
71 p
72 
73 b
74 e
75 t
76 w
77 e
78 e
79 n
80 
81 $
82 \
83 s
84 i
85 n
86 \
87 t
88 h
89 e
90 t
91 a
92 $
93 
94 a
95 n
96 d
97 
98 $
99 \
accuracy plain:  0.0
